In [1]:
import gc
from all_stand_var import conv_dict, lab_cols, vent_cols3
from all_own_funct import memory_downscale,memory_upscale,evaluate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale
import LR_build_CHD as pp
import tables
import datetime as dt


from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

locale.setlocale(locale.LC_ALL, 'fr_FR')
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_CHD_v3','1u_Results_no_mis')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import tables


df=pd.read_hdf(os.path.join('./Results_CHD/', 'processed_df.h5'),key='df',mode='r')
#df=memory_downscale(df)
print(df.info())



<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1469448 entries, (1, 7171037, Timestamp('2008-04-11 00:00:00')) to (789, 3536787, Timestamp('2020-01-10 00:00:00'))
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   pat_datetime     1469448 non-null  datetime64[ns]
 1   pat_weight_act   1469448 non-null  float16       
 2   mon_etco2        1457208 non-null  float16       
 3   mon_hr           1469448 non-null  float16       
 4   mon_ibp_mean     1469448 non-null  float16       
 5   mon_rr           1469448 non-null  float16       
 6   mon_sat          1469448 non-null  float16       
 7   vent_m_fio2      1469448 non-null  float16       
 8   vent_m_peep      1458648 non-null  float16       
 9   vent_m_ppeak     1458648 non-null  float16       
 10  vent_m_rr        1458648 non-null  float16       
 11  vent_m_tv_exp    1458648 non-null  float16       
 12  Age              1469448 n

In [3]:
from sklearn.preprocessing import StandardScaler
df=df.reset_index(drop=False)
df.info()
def index_1hr(group):
    if len(group) > 720:
        group=group.sort_values('pat_datetime')
        group=group.tail(720)
    group['tim']=pd.date_range(start='1/1/2018', periods=len(group), freq='1min')
    grouped = pd.Grouper(key='tim', freq='1H')
    group['idx_1hr']=group.groupby(grouped,sort=False).ngroup().add(1)
    #group['mis']=(group['pat_datetime'].max()-group['pat_datetime'].min()).total_seconds()/60
    del group['tim']
    return group
#grouped = pd.Grouper(key='pat_datetime', freq='60min',label='left',convention='start')
#df['idx_1hr']=df.groupby(['Adnum','pat_hosp_id','AdmissionDate','DischargeDate'],sort=False,as_index=False).ngroup().add(1)
df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
df['idx_1hr']=df['idx_1hr'].astype(str)
print(df['idx_1hr'].unique())
#df[['mis']]=StandardScaler().fit_transform(df[['mis']])
df=df.set_index(['pat_hosp_id','OK_datum'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469448 entries, 0 to 1469447
Data columns (total 20 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   level_0          1469448 non-null  int64         
 1   pat_hosp_id      1469448 non-null  int64         
 2   OK_datum         1469448 non-null  datetime64[ns]
 3   pat_datetime     1469448 non-null  datetime64[ns]
 4   pat_weight_act   1469448 non-null  float16       
 5   mon_etco2        1457208 non-null  float16       
 6   mon_hr           1469448 non-null  float16       
 7   mon_ibp_mean     1469448 non-null  float16       
 8   mon_rr           1469448 non-null  float16       
 9   mon_sat          1469448 non-null  float16       
 10  vent_m_fio2      1469448 non-null  float16       
 11  vent_m_peep      1458648 non-null  float16       
 12  vent_m_ppeak     1458648 non-null  float16       
 13  vent_m_rr        1458648 non-null  float16       
 14  ve

In [4]:
pat=pd.read_excel(r'Results_CHD\admissiondate_CHD0.xlsx',
                    parse_dates=['OK_datum'],index_col=0)
group=pat.groupby('pat_hosp_id',sort=False).max().reset_index()
group.drop_duplicates('pat_hosp_id',inplace=True)

df_train,df_val,df_test=pp.split_stratified_into_train_val_test(group, stratify_colname='Reintub',
                                         frac_train=0.7, frac_val=0.1, frac_test=0.2,
                                         random_state=1)

train_pat=df_train['pat_hosp_id'].unique()
test_pat=df_test['pat_hosp_id'].unique()
val_pat=df_val['pat_hosp_id'].unique()

df_train = df[df.index.get_level_values('pat_hosp_id').isin(train_pat)]
df_val = df[df.index.get_level_values('pat_hosp_id').isin(val_pat)]
df_test = df[df.index.get_level_values('pat_hosp_id').isin(test_pat)]
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1030725 entries, (7171037, Timestamp('2008-04-11 00:00:00')) to (3536787, Timestamp('2020-01-10 00:00:00'))
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   level_0          1030725 non-null  int64         
 1   pat_datetime     1030725 non-null  datetime64[ns]
 2   pat_weight_act   1030725 non-null  float16       
 3   mon_etco2        1022085 non-null  float16       
 4   mon_hr           1030725 non-null  float16       
 5   mon_ibp_mean     1030725 non-null  float16       
 6   mon_rr           1030725 non-null  float16       
 7   mon_sat          1030725 non-null  float16       
 8   vent_m_fio2      1030725 non-null  float16       
 9   vent_m_peep      1022805 non-null  float16       
 10  vent_m_ppeak     1022805 non-null  float16       
 11  vent_m_rr        1022805 non-null  float16       
 12  vent_m_tv_exp    1022805 non-null 

In [5]:
def y_modelling(df):
    y = df[['Reintub', 'Adnum', 'idx_1hr']]
    y = y.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['max'])
    y.columns = ["_".join(a) for a in y.columns.to_flat_index()]
    y.reset_index(drop=True, inplace=True, level='idx_1hr')
    y = y.reset_index().drop_duplicates().set_index(
        ['Adnum'])
    y = y[~y.index.duplicated(keep='last')]
    return y
Y_TRAIN=y_modelling(df_train)
Y_TEST=y_modelling(df_test)
Y_VAL=y_modelling(df_val)


In [6]:
print(Y_TRAIN.value_counts())
print(Y_VAL.value_counts())
print(Y_TEST.value_counts())
Y_TRAIN=Y_TRAIN['Reintub_max'].to_numpy()
Y_TEST=Y_TEST['Reintub_max'].to_numpy()
Y_VAL=Y_VAL['Reintub_max'].to_numpy()
Y_TRAIN=np.append(Y_TRAIN,Y_VAL)

Reintub_max
0.0            1477
1.0              90
dtype: int64
Reintub_max
0.0            209
1.0             13
dtype: int64
Reintub_max
0.0            410
1.0             27
dtype: int64


In [8]:
def x_modelling(df):
    temp = df[['Age', 'Adnum', 'idx_1hr','pat_weight_act','Diagnose']]
    df = df.drop(['Age','pat_weight_act','Extubation_date','level_0','pat_datetime', 'Reintub','Diagnose'], axis=1)
    df = df.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['mean', 'std'])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.stack().unstack([2, 1])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.reset_index().drop_duplicates().set_index('Adnum')
    df = df.fillna(method='ffill').fillna(method='bfill')

    temp = temp.groupby('Adnum', sort=False).agg(['mean'])
    temp.columns = ["_".join(a) for a in temp.columns.to_flat_index()]
    temp = temp.reset_index().drop_duplicates().set_index('Adnum')
    temp = temp[~temp.index.duplicated(keep='last')]
    df = df.merge(temp, right_index=True, left_index=True, how='left')
    return df
X_TRAIN=memory_upscale(x_modelling(df_train))
X_TEST=memory_upscale(x_modelling(df_test))
X_VAL=memory_upscale(x_modelling(df_val))
print(X_TRAIN.info())
print(X_TEST.info())
X_TRAIN=X_TRAIN.fillna(method='ffill').fillna(method='bfill')
X_VAL=X_VAL.fillna(method='ffill').fillna(method='bfill')
X_TEST=X_TEST.fillna(method='ffill').fillna(method='bfill')

X_TRAIN=X_TRAIN.append(X_VAL)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 1484 to 1482
Columns: 243 entries, mon_etco2_mean_1 to Diagnose_mean
dtypes: float64(242), int64(1)
memory usage: 2.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 437 entries, 1483 to 1473
Columns: 243 entries, mon_etco2_mean_1 to Diagnose_mean
dtypes: float64(241), int64(2)
memory usage: 849.2 KB
None


In [ ]:
f = open(os.path.join(output_folder, 'x_train.txt'), 'wb')
pickle.dump(X_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'x_test.txt'), 'wb')
pickle.dump(X_TEST, f)
f.close()

f = open(os.path.join(output_folder, 'y_train.txt'), 'wb')
pickle.dump(Y_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'y_test.txt'), 'wb')
pickle.dump(Y_TEST, f)
f.close()

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_TRAIN, Y_TRAIN)




Fitting 3 folds for each of 20 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.6min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 14, 18, 22, 26, 30,
                                                      34, 38, 42, 46, 50,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [10]:
print(X_TRAIN.info())
print(X_TEST.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1789 entries, 1484 to 1481
Columns: 243 entries, mon_etco2_mean_1 to Diagnose_mean
dtypes: float64(242), int64(1)
memory usage: 3.3 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 437 entries, 1483 to 1473
Columns: 243 entries, mon_etco2_mean_1 to Diagnose_mean
dtypes: float64(241), int64(2)
memory usage: 849.2 KB
None


In [11]:
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42,max_depth=10)
base_model.fit(X_TRAIN, Y_TRAIN)
base_accuracy = evaluate(base_model, X_TEST, Y_TEST,'base_accuracy RF',output_folder)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_TEST,Y_TEST,'Best_random RF',output_folder)

Model Performance
average_precision = 0.2806697959721222
F1 score: 0.13333333333333333 
AUC = 0.7817524841915087
Model Performance
average_precision = 0.4016124888026592
F1 score: 0.07142857142857142 
AUC = 0.8523486901535682


In [12]:
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import cm

# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

Brier score losses: (the smaller the better)
No calibration: 0.048
With isotonic calibration: 0.049
With sigmoid calibration: 0.048


In [13]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
from matplotlib.backends.backend_pdf import PdfPages

from all_own_funct import roc_auc_ci
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_alls.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_all.pdf"))
    os.remove(os.path.join(output_folder,f"Result_scores_all.txt"))
clf = best_random 


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

# write scoring metrics to file
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of random forrest")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))
a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)


plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)

plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Random Forrest hour data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)



Confidence interval for the score: [0.780 - 0.912]
(0.7606475223472423, 0.944049857959894)


In [14]:
# Number of trees in random forest
# Number of features to consider at every split
solver =['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# Maximum number of levels in tree
class_weight=[{0:0.1,1:0.9},'balanced']
# Minimum number of samples required to split a node
penalty=['l1', 'l2', 'elasticnet']
# Minimum number of samples required at each leaf node
C=np.logspace(-3,3,7)
# Method of selecting samples for training each tree
# Create the random grid
random_grid = {'solver':solver,
                'class_weight':class_weight,
                'penalty':penalty,
                'C':C}

# First create the base model to tune
lr = LogisticRegression(max_iter=1000)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lr_random = RandomizedSearchCV(estimator = lr , param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
lr_random.fit(X_TRAIN, Y_TRAIN)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:   36.3s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.0min finished


RandomizedSearchCV(cv=3, estimator=LogisticRegression(max_iter=1000), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'class_weight': [{0: 0.1, 1: 0.9},
                                                         'balanced'],
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=42, verbose=2)

In [15]:
base_model_lr = LogisticRegression(max_iter=1000)
base_model_lr.fit(X_TRAIN, Y_TRAIN)
base_accuracy_lr = evaluate(base_model_lr, X_TEST, Y_TEST,'base LR',output_folder)

best_random_lr = lr_random.best_estimator_
random_accuracy_lr = evaluate(best_random_lr, X_TEST,Y_TEST,'Best Random LR',output_folder)


# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random_lr.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random_lr, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random_lr, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)
    

C:\Users\berend\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
Model Performance
average_precision = 0.27482673146387465
F1 score: 0.15384615384615383 
AUC = 0.7913279132791327
Model Performance
average_precision = 0.15925805213079813
F1 score: 0.2564102564102564 
AUC = 0.6233965672990063
Brier score losses: (the smaller the better)
No calibration: 0.118
With isotonic calibration: 0.056
With sigmoid calibration: 0.058


In [16]:
clf = best_random_lr


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of logistic regression")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)


n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))

a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)

plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)


plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Logistic Regression data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)
pdf.close()


Confidence interval for the score: [0.507 - 0.731]
(0.5073920628052035, 0.739401071792809)


In [17]:
import pickle
f = open(os.path.join(output_folder,'ran_for.sav'), 'wb')
pickle.dump(best_random, f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'wb')
pickle.dump(best_random_lr, f)
f.close()